In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import rasterio
from functools import partial
from sklearn.model_selection import train_test_split
import tensorflow.keras.applications as app

In [3]:
df = pd.read_csv('/datasets/rpartsey/satellite/planet/2017-su-au-1_2017-sp-1_2017-sp-2_256x256_no_black_small.csv')
df['image_path'] = df['image']
df['label'] = df['mask_exists'].astype(np.uint8)
df = df[['image_path', 'label']]

In [4]:
train_df_pos, val_df_pos = train_test_split(df[df['label'] == 1], test_size=100)
train_df_neg, val_df_neg = train_test_split(df[df['label'] == 0], test_size=100)

train_df = pd.concat([train_df_pos, train_df_neg]).sample(frac=1)
val_df = pd.concat([val_df_pos, val_df_neg]).sample(frac=1)

In [5]:
train_df.shape

(4720, 2)

In [6]:
val_df.shape

(200, 2)

In [7]:
def read_tif(file_path):
    with rasterio.open(file_path) as src:
        data = src.read()
    return data

def get_rgb(image):
    b, g, r, nir = image
    return np.array([r, g, b])

def dataframe_reader(df):
    for index, row in df.iterrows():
        yield read_tif(row.image_path), row.label

def dataset_generator(df):
    for image, label in dataframe_reader(df):
        yield get_rgb(image), label

val_gen = partial(dataset_generator, df=val_df)
train_gen = partial(dataset_generator, df=train_df)

In [8]:
def normalize_image(image, label):
    return image / tf.uint16.max, label 

def transpose_channels(image, label):
    return tf.transpose(image, perm=[1, 2, 0]), label

In [9]:
num_epochs = 20

In [10]:
val_dataset = tf.data.Dataset.from_generator(
    val_gen,
    (tf.uint16, tf.uint8),
    ((tf.TensorShape([3, 256, 256]), tf.TensorShape([])))
)
val_dataset = val_dataset.map(transpose_channels)
val_dataset = val_dataset.map(normalize_image)
# val_dataset = val_dataset.repeat(num_epochs)
val_dataset

<DatasetV1Adapter shapes: ((256, 256, 3), ()), types: (tf.float32, tf.uint8)>

In [11]:
train_dataset = tf.data.Dataset.from_generator(
    val_gen,
    (tf.uint16, tf.uint8),
    ((tf.TensorShape([3, 256, 256]), tf.TensorShape([])))
)
train_dataset = train_dataset.map(transpose_channels)
train_dataset = train_dataset.map(normalize_image)
train_dataset = train_dataset.repeat(num_epochs)
train_dataset

<DatasetV1Adapter shapes: ((256, 256, 3), ()), types: (tf.float32, tf.uint8)>

In [15]:
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 1000

train_batches = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
validation_batches = val_dataset.batch(BATCH_SIZE)

In [16]:
IMG_SHAPE = (256, 256, 3)

base_model = app.VGG16(input_shape=IMG_SHAPE,
                       include_top=False,
                       weights='imagenet')

In [17]:
base_model.trainable = False

In [18]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0     

In [19]:
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.Flatten(name='flatten'),
    tf.keras.layers.Dense(4096, activation='relu', name='fc1'),
    tf.keras.layers.Dense(4096, activation='relu', name='fc2'),
    tf.keras.layers.Dense(1, activation='softmax', name='predictions')
])

In [20]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 8, 8, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 32768)             0         
_________________________________________________________________
fc1 (Dense)                  (None, 4096)              134221824 
_________________________________________________________________
fc2 (Dense)                  (None, 4096)              16781312  
_________________________________________________________________
predictions (Dense)          (None, 1)                 4097      
Total params: 165,721,921
Trainable params: 151,007,233
Non-trainable params: 14,714,688
_________________________________________________________________


In [22]:
loss0,accuracy0 = model.evaluate(validation_batches, steps = None)

7/7 [==============================] - 14s 2s/step - loss: 0.7999 - accuracy: 0.5000


In [34]:
print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))

initial loss: 0.83
initial accuracy: 0.50


In [ ]:
history = model.fit(train_batches,
                    epochs=num_epochs,
                    validation_data=validation_batches)

Epoch 1/20
     15/Unknown - 38s 3s/step - loss: 0.8070 - accuracy: 0.5063

In [14]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


In [25]:
# tf.debugging.set_log_device_placement(True)

try:
  # Specify an invalid GPU device
  with tf.device('/device:GPU:1'):
        history = model.fit(train_batches,
                    epochs=num_epochs,
                    validation_data=validation_batches)
except RuntimeError as e:
    print(e)


Epoch 1/20
      1/Unknown - 11s 11s/step

UnknownError: 2 root error(s) found.
  (0) Unknown:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node sequential/vgg16/block1_conv1/Conv2D (defined at /home/rpartsey/.virtualenvs/change_detection/lib/python3.6/site-packages/tensorflow_core/python/framework/ops.py:1751) ]]
	 [[metrics/accuracy/Cast_4/_80]]
  (1) Unknown:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node sequential/vgg16/block1_conv1/Conv2D (defined at /home/rpartsey/.virtualenvs/change_detection/lib/python3.6/site-packages/tensorflow_core/python/framework/ops.py:1751) ]]
0 successful operations.
0 derived errors ignored. [Op:__inference_distributed_function_1877]

Function call stack:
distributed_function -> distributed_function
